In [ ]:
!pip install tmdbv3api

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import requests
import urllib.request
import bs4 as bs

In [ ]:
# using tmbd website for metadata of the movies
from tmdbv3api import TMDb
import json
import requests

tmdb = TMDb()
tmdb.api_key = 'ef8ca48cd6c13b4f6428be8546a62fb4'

# Functions used to extract the data from TMDb

These functions are used in "Preprocessing 2" file already. Copied here! Positioned above all for code readability and neatness.

In [ ]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        return np.NaN

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

# Extracting features of 2020 movies from Wikipedia

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [ ]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [ ]:
tables = soup.find_all('table',class_='wikitable sortable')

In [ ]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] 
# avoided "ValueError: invalid literal for int() with base 10: '1"'

In [ ]:
# concating the data above
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df

In [ ]:
df_2020 = df.loc[:, ['Title','Cast and crew']]
df_2020

# Errors fixed

### I observed some errors while using the get_genre() function and found it at which index the error is generating with the help of following function before

In [ ]:
"""
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_movieid(movies):
    movie_ids=[]
    count = 0
    for x in movies:
        count += 1
        result = tmdb_movie.search(str(x))
        try:
            movie_ids.append(result[0].id)
        except:
            print("error occured at: ", count)
            movie_ids.append(np.NaN)
            continue
    return movie_ids


ids = get_movieid(df_2020['Title'])

"""

"""
The error occured at 
1. at 54th index - ('No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic')
2. at 126th index - ('The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)') movie is not recognized in TMDB

To fix the errors, I have updated movie name at 126th index, dropped 54th index data, and reset index

"""

df_2020.loc[126, 'Title']
# output: 'The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)'

df_2020.loc[126, 'Title'] = 'The SpongeBob Movie: Sponge on the Run'

df_2020.loc[54, 'Title']
# output: 'No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic'

df_2020.drop(df_2020.index[54], inplace=True)

df_2020.reset_index(drop=True, inplace=True)

# Continuing extracting the following

In [ ]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2020

In [ ]:
# renamed column
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [ ]:
# converting movie name to lowercase
df_2020['movie_title'] = df_2020['movie_title'].str.lower()

In [ ]:
# filtering the data
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
# handling nan values of actor names
new_df20['actor_2_name'] = new_df20['actor_2_name'].replace(np.nan, 'unknown')
new_df20['actor_3_name'] = new_df20['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:
# dropping null rows
new_df20.dropna(how='any', inplace=True)

In [ ]:
# final data
new_df20

# Extracting features of 2021 movies from Wikipedia

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [ ]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [ ]:
tables = soup.find_all('table',class_='wikitable sortable')

In [ ]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] 
# avoided "ValueError: invalid literal for int() with base 10: '1"'

In [ ]:
# concating the data above
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df

In [ ]:
df_2021 = df.loc[:, ['Title','Cast and crew']]
df_2021

In [ ]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2021

In [ ]:
# renamed column
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [ ]:
# converting movie name to lowercase
df_2021['movie_title'] = df_2021['movie_title'].str.lower()

In [ ]:
# filtering the data
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
# handling nan values of actor names
new_df21['actor_2_name'] = new_df21['actor_2_name'].replace(np.nan, 'unknown')
new_df21['actor_3_name'] = new_df21['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:
# dropping null rows
new_df21.dropna(how='any', inplace=True)

In [ ]:
# final dataset
new_df21

# Merging with movies data upto 2019

In [ ]:
my_df = pd.concat([new_df20, new_df21],ignore_index=True)

In [ ]:
old_df = pd.read_csv('../datasets/movie_upto_2019.csv')

In [ ]:
final_df = pd.concat([my_df, old_df], ignore_index=True)

In [ ]:
final_df

In [ ]:
# saving the dataset
final_df.to_csv('../datasets/movies_upto_2021.csv', index=False)